Importing all the required libraries for EDA

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as stats
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

import os
%matplotlib inline




In [269]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns',5000)


Importing all the files as shown below

campaign_data.csv: Campaign information for each of the 28 campaigns

coupon_item_mapping.csv: Mapping of coupon and items valid for discount under that coupon

customer_demographics.csv: Customer demographic information for some customers

customer_transaction_data.csv: Transaction data for all customers for duration of campaigns in the train data

In [270]:
campaign_data=pd.read_csv("D:\Downloads\DigitalMarketing\predicting-coupon-redemption\campaign_data.csv", decimal = ',')
customer_demographics=pd.read_csv("D:\Downloads\DigitalMarketing\predicting-coupon-redemption\customer_demographics.csv", decimal = ',')
customer_transaction_data= pd.read_csv("D:\Downloads\DigitalMarketing\predicting-coupon-redemption\customer_transaction_data.csv", decimal = ',')
item_data= pd.read_csv("D:\Downloads\DigitalMarketing\predicting-coupon-redemption\item_data.csv")
coupon_item_mapping=pd.read_csv("D:\Downloads\DigitalMarketing\predicting-coupon-redemption\coupon_item_mapping.csv", decimal = ',')


In [271]:
train_data=pd.read_csv('train2.csv')

In [273]:
type(train_data)

pandas.core.frame.DataFrame

Checking for any null values

In [274]:
campaign_data.isnull().values.any()


False

In [275]:
customer_transaction_data.isnull().values.any()

False

In [276]:
item_data.isnull().values.any()

False

In [277]:
coupon_item_mapping.isnull().values.any()

False

In [278]:
customer_demographics.isnull().values.any()

True

There are a few null values- so checking the columns which contain null

In [279]:
customer_demographics.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [280]:
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [281]:
customer_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
customer_id       760 non-null int64
age_range         760 non-null object
marital_status    431 non-null object
rented            760 non-null int64
family_size       760 non-null object
no_of_children    222 non-null object
income_bracket    760 non-null int64
dtypes: int64(3), object(4)
memory usage: 41.6+ KB


Replacing the null values 

In [282]:
customer_demographics.marital_status.value_counts()

Married    317
Single     114
Name: marital_status, dtype: int64

In [283]:
customer_demographics.family_size.value_counts()

2     303
1     248
3     104
5+     57
4      48
Name: family_size, dtype: int64

In [284]:
customer_demographics.no_of_children.value_counts()

1     107
3+     60
2      55
Name: no_of_children, dtype: int64

To replace the + sign

In [285]:
customer_demographics['family_size'] = customer_demographics.family_size.apply(lambda x: int(re.sub('\+','',x)))
customer_demographics['no_of_children'] = customer_demographics.no_of_children.apply(lambda x: int(re.sub('\+','',x)) if pd.notna(x) else x)

We do this do that- it's in a format where we can manipulate

Filling NaN values for marital_status

In [286]:
customer_demographics.loc[pd.isnull(customer_demographics.marital_status) & (customer_demographics.family_size == 1),'marital_status'] = 'Single'

In [287]:
customer_demographics.loc[(customer_demographics.family_size - customer_demographics.no_of_children == 1) & pd.isnull(customer_demographics.marital_status),'marital_status'] = 'Single'


In [288]:
customer_demographics.loc[(pd.isnull(customer_demographics.marital_status)) & ((customer_demographics.family_size - customer_demographics.no_of_children) == 2)  
              & (pd.notnull(customer_demographics.no_of_children)),'marital_status'] = 'Married'

In [289]:
customer_demographics.loc[pd.isnull(customer_demographics.marital_status) & (pd.isnull(customer_demographics.no_of_children)) 
              & (customer_demographics.family_size ==2),'marital_status'] = 'Married'

In [290]:
a = customer_demographics.marital_status.groupby((customer_demographics.family_size - customer_demographics.no_of_children) == 2).value_counts()
print(a[True])

marital_status
Married    186
Single      10
Name: marital_status, dtype: int64


In [291]:
customer_demographics.marital_status.isnull().sum()

0

All missing values have been meaningfully replaced

In [292]:
customer_demographics.isnull().sum()

customer_id         0
age_range           0
marital_status      0
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [293]:
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,Married,0,3,1.0,3
3,8,26-35,Married,0,4,2.0,6
4,10,46-55,Single,0,1,NaN,5


Filling NaN values for children

In [294]:
customer_demographics.loc[pd.isnull(customer_demographics.no_of_children) & (customer_demographics.marital_status == 'Married') & (customer_demographics.family_size == 2),'no_of_children'] = 0

In [295]:
customer_demographics.loc[pd.isnull(customer_demographics.no_of_children) & (customer_demographics.family_size == 1), 'no_of_children'] = 0

In [296]:
customer_demographics.loc[pd.isnull(customer_demographics.no_of_children) & (customer_demographics.family_size == 2),'no_of_children'] = 1

In [297]:

customer_demographics['no_of_children']=customer_demographics['no_of_children'].astype(np.int64)

In [298]:
customer_demographics.no_of_children.isnull().sum()

0

In [299]:
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,0,4
1,6,46-55,Married,0,2,0,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,0,5


In [300]:
customer_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
customer_id       760 non-null int64
age_range         760 non-null object
marital_status    760 non-null object
rented            760 non-null int64
family_size       760 non-null int64
no_of_children    760 non-null int64
income_bracket    760 non-null int64
dtypes: int64(5), object(2)
memory usage: 41.6+ KB


In [301]:
customer_demographics["marital_status"] = encoder.fit_transform(customer_demographics["marital_status"])

In [302]:
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,0,0,2,0,4
1,6,46-55,0,0,2,0,5
2,7,26-35,0,0,3,1,3
3,8,26-35,0,0,4,2,6
4,10,46-55,1,0,1,0,5


In [303]:
customer_demographics["age_range"] = encoder.fit_transform(customer_demographics["age_range"])

In [304]:
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,5,0,0,2,0,4
1,6,3,0,0,2,0,5
2,7,1,0,0,3,1,3
3,8,1,0,0,4,2,6
4,10,3,1,0,1,0,5


In [305]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [306]:
campaign_data.campaign_type.value_counts()

Y    22
X     6
Name: campaign_type, dtype: int64

In [307]:
campaign_data["campaign_type"] = encoder.fit_transform(campaign_data.campaign_type)

Getting the date to a suitable format

In [308]:
campaign_data['start_date'] = pd.to_datetime(campaign_data['start_date'], format = '%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data['end_date'], format = '%d/%m/%y')

Calcullating the duration of the campaign

In [309]:
campaign_data["campaign_duration"] = campaign_data["end_date"] - campaign_data["start_date"]
campaign_data["campaign_duration"] = campaign_data["campaign_duration"].apply(lambda x: x.days)

In [310]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date,campaign_duration
0,24,1,2013-10-21,2013-12-20,60
1,25,1,2013-10-21,2013-11-22,32
2,20,1,2013-09-07,2013-11-16,70
3,23,1,2013-10-08,2013-11-15,38
4,21,1,2013-09-16,2013-10-18,32


In [311]:
campaign_data

,campaign_id,campaign_type,start_date,end_date,campaign_duration
0,24,1,2013-10-21,2013-12-20,60
1,25,1,2013-10-21,2013-11-22,32
2,20,1,2013-09-07,2013-11-16,70
3,23,1,2013-10-08,2013-11-15,38
4,21,1,2013-09-16,2013-10-18,32
5,22,0,2013-09-16,2013-10-18,32
6,18,0,2013-08-10,2013-10-04,55
7,19,1,2013-08-26,2013-09-27,32
8,17,1,2013-07-29,2013-08-30,32
9,16,1,2013-07-15,2013-08-16,32


Dealing with Customer Transaction Dataframe

In [312]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.5,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.0,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [313]:
customer_transaction_data['date'] = pd.to_datetime(customer_transaction_data['date'])
customer_transaction_data = customer_transaction_data.sort_values('date').reset_index(drop=True)

In [314]:
customer_transaction_data['day'] = customer_transaction_data["date"].apply(lambda x: x.day)
customer_transaction_data['dow'] = customer_transaction_data["date"].apply(lambda x: x.weekday())
customer_transaction_data['month'] = customer_transaction_data["date"].apply(lambda x: x.month)

In [315]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,day,dow,month
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,0,1
1,2012-01-02,464,20697,1,92.26,-21.37,-35.62,2,0,1
2,2012-01-02,464,20717,2,28.5,-27.78,0.0,2,0,1
3,2012-01-02,464,21008,1,35.26,-17.81,0.0,2,0,1
4,2012-01-02,464,22243,2,118.97,-22.8,0.0,2,0,1


Checking if the coupon is used on not- based on the custoemer transaction data- based on the coupon discount. If the discount is greater than 1 then Coupon is availed

In [319]:
customer_transaction_data['coupon_used'] = customer_transaction_data.coupon_discount.apply(lambda x: 1 if x !=0 else 0)

In [320]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,day,dow,month,coupon_used
0,2012-01-02,1501,26830,1,45.950,-10.69,0.00,2,0,1,0
1,2012-01-02,464,20697,1,113.630,-21.37,-35.62,2,0,1,1
2,2012-01-02,464,20717,2,28.140,-13.89,0.00,2,0,1,0
3,2012-01-02,464,21008,1,53.070,-17.81,0.00,2,0,1,0
4,2012-01-02,464,22243,2,70.885,-11.40,0.00,2,0,1,0


Dealing with items data frame

In [321]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [322]:
item_data.brand_type.value_counts()

Established    62842
Local          11224
Name: brand_type, dtype: int64

Getting more insights from the data as shown above - getting to know the brand

In [323]:
item_data.category.value_counts()

Grocery                   32448
Pharmaceutical            24471
Natural Products           2533
Dairy, Juices & Snacks     2425
Skin & Hair Care           2244
Meat                       2080
Packaged Meat              1966
Prepared Food              1880
Bakery                     1679
Seafood                     728
Flowers & Plants            664
Miscellaneous               385
Alcohol                     303
Garden                      102
Restauarant                  78
Salads                       40
Travel                       19
Fuel                         14
Vegetables (cut)              7
Name: category, dtype: int64

In [324]:
item_data.brand_type = encoder.fit_transform(item_data["brand_type"])
item_data.category = encoder.fit_transform(item_data["category"])

In [325]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,0,6
1,2,1,0,8
2,3,56,1,1
3,4,56,1,6
4,5,56,1,6


Dealing with Coupon Dataframe

In [327]:
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


Now everything in the correct format

Merge coupon item data and items data on item_id

In [328]:
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [329]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,0,6
1,2,1,0,8
2,3,56,1,1
3,4,56,1,6
4,5,56,1,6


In [330]:
coupons_items = pd.merge(coupon_item_mapping, item_data, on="item_id", how="left")

In [331]:
coupons_items.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,1,6
1,107,75,56,1,6
2,494,76,209,0,6
3,522,77,278,0,6
4,518,77,278,0,6


Aggregate the customer transaction by 'item_id'

In [332]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,day,dow,month,coupon_used
0,2012-01-02,1501,26830,1,45.950,-10.69,0.00,2,0,1,0
1,2012-01-02,464,20697,1,113.630,-21.37,-35.62,2,0,1,1
2,2012-01-02,464,20717,2,28.140,-13.89,0.00,2,0,1,0
3,2012-01-02,464,21008,1,53.070,-17.81,0.00,2,0,1,0
4,2012-01-02,464,22243,2,70.885,-11.40,0.00,2,0,1,0


In [333]:
type(customer_transaction_data)

pandas.core.frame.DataFrame

In [334]:
type(customer_transaction_data.quantity)

pandas.core.series.Series

In [335]:
customer_transaction_data.dtypes

date               datetime64[ns]
customer_id                 int64
item_id                     int64
quantity                    int64
selling_price             float64
other_discount            float64
coupon_discount           float64
day                         int64
dow                         int64
month                       int64
coupon_used                 int64
dtype: object

In [337]:
customer_transaction_data.dtypes

date               datetime64[ns]
customer_id                 int64
item_id                     int64
quantity                    int64
selling_price             float64
other_discount            float64
coupon_discount           float64
day                         int64
dow                         int64
month                       int64
coupon_used                 int64
dtype: object

In [409]:
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,5,0,0,2,0,4
1,6,3,0,0,2,0,5
2,7,1,0,0,3,1,3
3,8,1,0,0,4,2,6
4,10,3,1,0,1,0,5


All data is cleaned and data is made sense of

In [267]:
train.to_csv('trainFinal.csv')